# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
#from sentiment_analysis_spanish import sentiment_analysis
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import glob
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


## Ventas, GB1 & Numero de Subclases

In [3]:
def generate_months(start_year, end_year):
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            months.append(f"{year:04d}-{month:02d}")
    return months

start_year = 2022
end_year = 2023

months_list = generate_months(start_year, end_year)
months_list

['2022-01',
 '2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12',
 '2023-01',
 '2023-02',
 '2023-03',
 '2023-04',
 '2023-05',
 '2023-06',
 '2023-07',
 '2023-08',
 '2023-09',
 '2023-10',
 '2023-11',
 '2023-12']

In [4]:
def generate_months(start_year, end_year):
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            months.append(f"{year:04d}-{month:02d}")
    return months

start_year = 2023
end_year = 2023

months_list_2 = generate_months(start_year, end_year)
months_list_2

['2023-01',
 '2023-02',
 '2023-03',
 '2023-04',
 '2023-05',
 '2023-06',
 '2023-07',
 '2023-08',
 '2023-09',
 '2023-10',
 '2023-11',
 '2023-12']

In [5]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        GRO.GROUP_NAME,
        DEPS.DEPT_NAME,
        CLA.CLASE,
        CLA.CLASS_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.DEPS AS DEPS ON DEPS.DEPT = IM.DEPT
        INNER JOIN MSTRDB.DWH.GROUPS AS GRO ON GRO.GROUP_NO = DEPS.GROUP_NO
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') AS PERIODO,
    FV.TIEM_DIA_ID,
    PROMOS.INICIO,
    PROMOS.FIN,
    PROMOS.TIPO_OFERTA_ID,
    PROMOS.TIPO_OFERTA_DESC,
    PROMOS.GROUP_NAME,
    PROMOS.DEPT_NAME,
    IM.CLASE,
    CLA.CLASS_NAME,
    LGL.GEOG_LOCL_COD,
    FV.VNTA_IMPORTE_SIN_IVA AS VENTA_SIN_IVA,
    FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND AS COSTO,
    VENTA_SIN_IVA - COSTO AS GB1
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

df_1 = pd.DataFrame()
for month in months_list:
    cursor.execute(query.format(periodo_query = month))
    df_aux = cursor.fetch_pandas_all()
    df_1 = pd.concat([df_aux, df_1])
df_1.head(2)

df_2 = df_1.groupby(['PERIODO', 'TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'GROUP_NAME', 'DEPT_NAME', 'CLASE', 'CLASS_NAME'])['VENTA_SIN_IVA', 'GB1'].sum().reset_index()
for i in ['VENTA_SIN_IVA', 'GB1']:
    df_2[i] = df_2[i].astype(int)

df_2['GB1_%'] = round(df_2['GB1'] / df_2['VENTA_SIN_IVA'] * 100, 1)

df_2.head(2)

C:\Users\leonardo.mangold\AppData\Local\Temp\ipykernel_4360\2856392385.py:67: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2 = df_1.groupby(['PERIODO', 'TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'GROUP_NAME', 'DEPT_NAME', 'CLASE', 'CLASS_NAME'])['VENTA_SIN_IVA', 'GB1'].sum().reset_index()


,PERIODO,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,INICIO,FIN,GROUP_NAME,DEPT_NAME,CLASE,CLASS_NAME,VENTA_SIN_IVA,GB1,GB1_%
0,2022-01,10,Imbatibles,2022-01-05,2022-01-24,BEBIDAS,BEB. ALCOHOLICAS,10002,CERVEZAS,6424483,928701,14.5
1,2022-01,10,Imbatibles,2022-01-05,2022-01-24,BEBIDAS,BEB. ALCOHOLICAS,10004,DESTILADAS,65143,8134,12.5


In [8]:
df_2.to_excel('Ventas y GB1.xlsx', index = False)

In [7]:
sys.exit()

NameError: name 'sys' is not defined

## Cantidad de Articulos

### Queries

In [ ]:
query = '''

'''

cursor.execute(query.format(periodo_query = periodo))
cant_1 = cursor.fetch_pandas_all()
cant_1['REF'] = 'Venta Promo'
print(cant_1.shape[0])
cant_1.head(2)

In [ ]:
# Para comparar con las ventas de la subclase o clase, tengo que agrupar y perder el tipo de oferta
cant_2 = cant_1.groupby(['REF', 'CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['UNIDADES_PROMO'].sum().reset_index()
cant_2.head(2)

In [ ]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        GRO.GROUP_NAME,
        DEPS.DEPT_NAME,
        CLA.CLASE,
        CLA.CLASS_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.DEPS AS DEPS ON DEPS.DEPT = IM.DEPT
        INNER JOIN MSTRDB.DWH.GROUPS AS GRO ON GRO.GROUP_NO = DEPS.GROUP_NO
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') AS PERIODO,
    FV.TIEM_DIA_ID,
    PROMOS.INICIO,
    PROMOS.FIN,
    PROMOS.TIPO_OFERTA_ID,
    PROMOS.TIPO_OFERTA_DESC,
    PROMOS.GROUP_NAME,
    PROMOS.DEPT_NAME,
    IM.CLASE,
    CLA.CLASS_NAME,
    LGL.GEOG_LOCL_COD,
    FV.VNTA_UNIDADES AS UNIDADES_TOTALES
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.CLASE = IM.CLASE AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

clases_1 = pd.DataFrame()
for month in months_list_2:
    cursor.execute(query.format(periodo_query = month))
    df_aux = cursor.fetch_pandas_all()
    clases = pd.concat([df_aux, clases_1])


#df_2 = df_1.groupby(['PERIODO', 'TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'GROUP_NAME', 'DEPT_NAME', 'CLASE', 'CLASS_NAME'])['VENTA_SIN_IVA', 'GB1'].sum().reset_index()
#for i in ['VENTA_SIN_IVA', 'GB1']:
    #df_2[i] = df_2[i].astype(int)

#df_2['GB1_%'] = round(df_2['GB1'] / df_2['VENTA_SIN_IVA'] * 100, 1)

#df_2.head(2)

clases_1.head(2)

In [ ]:
sub_cant_2 = sub_cant_1.groupby(['REF_2','CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['UNIDADES_TOTALES'].sum().reset_index()
sub_cant_2.head(2)

### tabla 6 - Num Articulos por Subclase

In [ ]:
e_4 = cant_2.merge(sub_cant_2, on = ['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'], how = 'inner')

e_4['RATIO_UNIDADES'] = round(e_4['UNIDADES_PROMO'] / e_4['UNIDADES_TOTALES'] * 100, 1)
for i in ['UNIDADES_PROMO', 'UNIDADES_TOTALES']:
    e_4[i] = e_4[i].astype(int)

e_4.head(2)

# Excel

In [ ]:
with pd.ExcelWriter(f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Promos {periodo}.xlsx", engine='xlsxwriter') as writer:
    e_1.to_excel(writer, sheet_name='Articulos Publicados', index=False) # tabla 1
    arts_2.to_excel(writer, sheet_name='Venta y GB1 por Promo', index=False) # tabla 2
    e_6.to_excel(writer, sheet_name='Numero de Subclases', index=False) # tabla 3
    e_2.to_excel(writer, sheet_name='Venta y GB1 por Subclase', index=False) # tabla 4
    e_3.to_excel(writer, sheet_name='Venta y GB1 Promo Agrupado', index=False) # tabla 5
    e_4.to_excel(writer, sheet_name='Num Articulos por Subclase', index=False) # tabla 6
    ace.to_excel(writer, sheet_name='Aceleracion', index=False) # tabla 7
    ace_2.to_excel(writer, sheet_name='Aceleracion por Subclase', index=False) # tabla 8
    ace_3.to_excel(writer, sheet_name='Aceleracion Resumen', index=False) # tabla 9
    e_5.to_excel(writer, sheet_name='Tickets', index=False) # tabla 10
    eventos.to_excel(writer, sheet_name='Numero de Eventos', index=False) # tabla 11
    precios.to_excel(writer, sheet_name=f"Precios {periodo_precios}", index=False) # tabla 12
    opt.to_excel(writer, sheet_name=f"OPT - Media Precios Compet", index=False) # tabla 13

# Fin